## Introduce

In [3]:
import tensorflow as tf
import numpy as np

In [4]:
x_data  = np.float32(np.random.randn(2, 100))
y_data = np.dot([0.100, 0.200], x_data) + 0.300

In [5]:
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = tf.matmul(W, x_data) + b

In [6]:
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

In [7]:
init = tf.global_variables_initializer()

In [8]:
sess = tf.Session()
sess.run(init)

In [10]:
for step in range(0, 201):
    sess.run(train)
    if step % 20 == 0:
        print (step, sess.run(W), sess.run(b))

0 [[0.27818656 0.16711512]] [0.3279436]
20 [[0.1 0.2]] [0.3]
40 [[0.1 0.2]] [0.3]
60 [[0.1 0.2]] [0.3]
80 [[0.1 0.2]] [0.3]
100 [[0.1 0.2]] [0.3]
120 [[0.1 0.2]] [0.3]
140 [[0.1 0.2]] [0.3]
160 [[0.1 0.2]] [0.3]
180 [[0.1 0.2]] [0.3]
200 [[0.1 0.2]] [0.3]


In [21]:
sess.close()

## intro_mnist

In [13]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../data/', one_hot=True)

Extracting ../data/train-images-idx3-ubyte.gz
Extracting ../data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


In [14]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [17]:
y_ = tf.placeholder("float", [None, 10])

In [18]:
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

In [19]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [20]:
init = tf.global_variables_initializer()

In [22]:
sess = tf.Session()
sess.run(init)

In [23]:
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [24]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [25]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [26]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9117


In [27]:
sess.close()

## Another try

In [28]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../data/')

Extracting ../data/train-images-idx3-ubyte.gz
Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


In [29]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [31]:
y_ = tf.placeholder(tf.int64, [None])

The raw formulation of cross-entropy,
`tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)), reduction_indices=[1]))`
can be numerically unstable.
So here we use `tf.losses.sparse_softmax_cross_entropy`on the raw
outputs of 'y', and then average across the batch

In [32]:
cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y_, logits=y)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [33]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [34]:
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [35]:
correct_prediction = tf.equal(tf.argmax(y, 1), y_)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9068


In [36]:
sess.close()

## Conv Net

In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../data/', one_hot=True)

Extracting ../data/train-images-idx3-ubyte.gz
Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


In [10]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [11]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [12]:
sess.run(tf.global_variables_initializer())

In [13]:
y = tf.nn.softmax(tf.matmul(x, W) + b)
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [14]:
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [15]:
correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Instructions for updating:
Use `argmax` instead
0.9141


****

In [16]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [17]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [18]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [19]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [20]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [21]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [22]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [23]:
cross_entropy = tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [24]:
import time
sess.run(tf.global_variables_initializer())
for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        begin_time = time.time()
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1],
                                                  keep_prob: 1.0})
        cost_time = time.time() - begin_time
        print("step: %d, training accuracy %g, time: %r" 
              % (i , train_accuracy, cost_time))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step: 0, training accuracy 0.1, time: 0.031227588653564453
step: 100, training accuracy 0.12, time: 0.015627145767211914
step: 200, training accuracy 0.16, time: 0.03125333786010742
step: 300, training accuracy 0.08, time: 0.015626192092895508
step: 400, training accuracy 0.08, time: 0.03125286102294922
step: 500, training accuracy 0.06, time: 0.01562356948852539
step: 600, training accuracy 0.08, time: 0.0312502384185791
step: 700, training accuracy 0.06, time: 0.01562643051147461
step: 800, training accuracy 0.06, time: 0.015629053115844727
step: 900, training accuracy 0.2, time: 0.01562643051147461
step: 1000, training accuracy 0.08, time: 0.01562786102294922
step: 1100, training accuracy 0.08, time: 0.03125357627868652
step: 1200, training accuracy 0.1, time: 0.031239748001098633
step: 1300, training accuracy 0.18, time: 0.01563239097595215
step: 1400, training accuracy 0.1, time: 0.015628814697265625
step: 1500, training accuracy 0.1, time: 0.03125715255737305
step: 1600, training

In [25]:
sess.run(tf.global_variables_initializer())
for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        begin_time = time.time()
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1],
                                                  keep_prob: 1.0})
        train_loss = cross_entropy.eval(feed_dict={x: batch[0], y_: batch[1],
                                                  keep_prob: 1.0})
        cost_time = time.time() - begin_time
        print("step: %d, training loss %g ,training accuracy %g, time: %r" 
              % (i ,train_loss, train_accuracy, cost_time))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step: 0, training loss -341.94 ,training accuracy 0.12, time: 0.05914950370788574
step: 100, training loss nan ,training accuracy 0.02, time: 0.0451200008392334
step: 200, training loss nan ,training accuracy 0.14, time: 0.041109323501586914
step: 300, training loss nan ,training accuracy 0.1, time: 0.047153472900390625
step: 400, training loss nan ,training accuracy 0.08, time: 0.0451204776763916
step: 500, training loss nan ,training accuracy 0.12, time: 0.043616533279418945
step: 600, training loss nan ,training accuracy 0.18, time: 0.04261302947998047
step: 700, training loss nan ,training accuracy 0.1, time: 0.04211235046386719
step: 800, training loss nan ,training accuracy 0.1, time: 0.04812812805175781
step: 900, training loss nan ,training accuracy 0.14, time: 0.047124624252319336
step: 1000, training loss nan ,training accuracy 0.1, time: 0.04662442207336426
step: 1100, training loss nan ,training accuracy 0.1, time: 0.045621395111083984
step: 1200, training loss nan ,trainin

In [31]:
print("test accuracy %g" % accuracy.eval(feed_dict={x: mnist.test.images,
                                                  y_: mnist.test.labels,
                                                  keep_prob: 1.0}))

test accuracy 0.098


**Cosidering the Softmax-Loss Numerical Stability**

In [45]:
mnist = input_data.read_data_sets('../data/')
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("int64", shape=[None])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

# y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
# cross_entropy = tf.reduce_sum(y_ * tf.log(y_conv))
cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y_, logits=y_conv)

Extracting ../data/train-images-idx3-ubyte.gz
Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


In [46]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv), tf.argmax(y_))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [ ]:
sess.run(tf.global_variables_initializer())
for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        begin_time = time.time()
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1],
                                                  keep_prob: 1.0})
        train_loss = cross_entropy.eval(feed_dict={x: batch[0], y_: batch[1],
                                                  keep_prob: 1.0})
        cost_time = time.time() - begin_time
        print("step: %d, training loss %g ,training accuracy %g, time: %r" 
              % (i ,train_loss, train_accuracy, cost_time))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step: 0, training loss 8.12807 ,training accuracy 0, time: 0.1348741054534912
step: 100, training loss 0.567834 ,training accuracy 0.1, time: 0.05263996124267578
step: 200, training loss 0.340762 ,training accuracy 0, time: 0.047626495361328125
step: 300, training loss 0.238517 ,training accuracy 0, time: 0.04913067817687988
step: 400, training loss 0.310654 ,training accuracy 0.1, time: 0.05063438415527344
step: 500, training loss 0.181243 ,training accuracy 0, time: 0.049130916595458984
step: 600, training loss 0.328723 ,training accuracy 0, time: 0.051134586334228516
step: 700, training loss 0.286933 ,training accuracy 0, time: 0.04461932182312012
step: 800, training loss 0.0966255 ,training accuracy 0, time: 0.04411792755126953
step: 900, training loss 0.097214 ,training accuracy 0.1, time: 0.0416104793548584
step: 1000, training loss 0.100267 ,training accuracy 0, time: 0.04311490058898926
step: 1100, training loss 0.0521508 ,training accuracy 0, time: 0.04562115669250488
step: 12

##  Combine 

In [1]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
with tf.device('/gpu:0'):
    x = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.int64, [None])
    keep_prob = tf.placeholder(tf.float32)

In [3]:
def deepnn(x):
  """deepnn builds the graph for a deep net for classifying digits.

  Args:
    x: an input tensor with the dimensions (N_examples, 784), where 784 is the
    number of pixels in a standard MNIST image.

  Returns:
    A tuple (y, keep_prob). y is a tensor of shape (N_examples, 10), with values
    equal to the logits of classifying the digit into one of 10 classes (the
    digits 0-9). keep_prob is a scalar placeholder for the probability of
    dropout.
  """
  # Reshape to use within a convolutional neural net.
  # Last dimension is for "features" - there is only one here, since images are
  # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
  with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

  # First convolutional layer - maps one grayscale image to 32 feature maps.
  with tf.name_scope('conv1'):
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

  # Pooling layer - downsamples by 2X.
  with tf.name_scope('pool1'):
    h_pool1 = max_pool_2x2(h_conv1)

  # Second convolutional layer -- maps 32 feature maps to 64.
  with tf.name_scope('conv2'):
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

  # Second pooling layer.
  with tf.name_scope('pool2'):
    h_pool2 = max_pool_2x2(h_conv2)

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
  # is down to 7x7x64 feature maps -- maps this to 1024 features.
  with tf.name_scope('fc1'):
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
  with tf.name_scope('dropout'):
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Map the 1024 features to 10 classes, one for each digit
  with tf.name_scope('fc2'):
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
  return y_conv, keep_prob


def conv2d(x, W):
  """conv2d returns a 2d convolution layer with full stride."""
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
  """max_pool_2x2 downsamples a feature map by 2X."""
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)


def bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [4]:
mnist = input_data.read_data_sets('../data/')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
y_conv, keep_prob = deepnn(x)

In [6]:
with tf.name_scope('loss'):
    cross_entropy = tf.losses.sparse_softmax_cross_entropy(
        labels=y_, logits=y_conv)
cross_entropy = tf.reduce_mean(cross_entropy)

In [7]:
with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [8]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), y_)
    correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)

In [9]:
import tempfile
graph_location = tempfile.mkdtemp()
print('Saving graph to: %s' % graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(tf.get_default_graph())

Saving graph to: C:\Users\sun\AppData\Local\Temp\tmpn0zrkggm


### USE CPU

In [56]:
with tf.Session() as sess:
    begin_time = time.time()
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
      batch = mnist.train.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
      if i % 500 == 0:
        train_loss = cross_entropy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        during_time = time.time() - begin_time
        print('step %d, training loss %g, training accuracy %g, time: %r'
              % (i, train_loss, train_accuracy, during_time))
      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.12
step 0, training loss 8.78441, training accuracy 0.12, time: 0.34378671646118164
step 100, training accuracy 0.82
step 200, training accuracy 0.94
step 300, training accuracy 0.9
step 400, training accuracy 0.96
step 500, training accuracy 0.96
step 500, training loss 0.149972, training accuracy 0.96, time: 40.72666597366333
step 600, training accuracy 0.92
step 700, training accuracy 0.92
step 800, training accuracy 0.98
step 900, training accuracy 0.98
test accuracy 0.9614


### USE GPU

In [12]:
with tf.Session() as sess:
    begin_time = time.time()
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
      batch = mnist.train.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
      if i % 500 == 0:
        train_loss = cross_entropy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        during_time = time.time() - begin_time
        print('step %d, training loss %g, training accuracy %g, time: %r'
              % (i, train_loss, train_accuracy, during_time))
      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.18
step 0, training loss 5.33977, training accuracy 0.18, time: 0.186492919921875
step 100, training accuracy 0.76
step 200, training accuracy 0.82
step 300, training accuracy 0.88
step 400, training accuracy 0.92
step 500, training accuracy 0.94
step 500, training loss 0.174871, training accuracy 0.94, time: 3.2792015075683594
step 600, training accuracy 0.96
step 700, training accuracy 0.94
step 800, training accuracy 0.96
step 900, training accuracy 0.96
step 1000, training accuracy 0.92
step 1000, training loss 0.196284, training accuracy 0.92, time: 6.246205806732178
step 1100, training accuracy 1
step 1200, training accuracy 0.98
step 1300, training accuracy 0.98
step 1400, training accuracy 1
step 1500, training accuracy 0.96
step 1500, training loss 0.175489, training accuracy 0.96, time: 9.280698776245117
step 1600, training accuracy 0.98
step 1700, training accuracy 0.9
step 1800, training accuracy 0.98
step 1900, training accuracy 1
step 2000, tra

step 16600, training accuracy 1
step 16700, training accuracy 1
step 16800, training accuracy 1
step 16900, training accuracy 1
step 17000, training accuracy 1
step 17000, training loss 0.00230989, training accuracy 1, time: 102.37067031860352
step 17100, training accuracy 1
step 17200, training accuracy 1
step 17300, training accuracy 1
step 17400, training accuracy 1
step 17500, training accuracy 1
step 17500, training loss 0.00488085, training accuracy 1, time: 105.27428483963013
step 17600, training accuracy 1
step 17700, training accuracy 1
step 17800, training accuracy 1
step 17900, training accuracy 1
step 18000, training accuracy 1
step 18000, training loss 0.000489491, training accuracy 1, time: 108.28570342063904
step 18100, training accuracy 1
step 18200, training accuracy 1
step 18300, training accuracy 1
step 18400, training accuracy 1
step 18500, training accuracy 1
step 18500, training loss 0.000839726, training accuracy 1, time: 111.23501706123352
step 18600, training a